## Attractor Landscape analysis

This is an example of how Boolean network model was analyzed in our paper.

We provide the codes for 'Attractor landscape analysis', 'Perturbation analysis', and 'Analysis of network dynamics' (Please refer to the Method section).


In [ ]:
import numpy as np
import pandas as pd
import itertools
import networkx as nx
import copy
import os

from pyboolnet.file_exchange import bnet2primes, primes2bnet
from pyboolnet.interaction_graphs import primes2igraph
from pyboolnet.state_transition_graphs import primes2stg
from pyboolnet.attractors import compute_attractors_tarjan

from modules.attractorSim import rand_initial_states, compute_attractor_from_primes, compute_phenotype, Simulation
from modules.printlogic import print_node_logic

In [2]:
network_dir = './network/'
model_file = network_dir + 'EMT_Network.bnet'
primes = bnet2primes(model_file)
nodeList = list(primes.keys())
graph = primes2igraph(primes)
update_mode = "synchronous"

# nodes_order = sorted(primes.keys())  # 이미 선언된 상태여야 함
nodes_order = sorted(primes.keys())
print("총 노드 수:", len(nodes_order))
print("앞 40개 노드:", nodes_order[:40])

# (붙여넣기용) 자동 마커/phenotype 생성
# --- 위 2) 셀의 코드 전체를 여기에 붙여 넣으면 됨 ---

# --- 시작: phenotype/markers 자동 설정 (복붙해서 기존 phenotype 셀 대신 사용) ---
# nodes_order는 위에서 확인한 primes의 정렬된 노드 리스트

# 1) 마커 자동 찾기: E-cad (E-cadherin 계열)과 ZEB1 계열 노드 이름 검색
def find_node_by_keywords(nodes_order, keywords):
    for n in nodes_order:
        lower = n.lower()
        for kw in keywords:
            if kw.lower() in lower:
                return n
    return None

ecad_candidates = ["ecad", "cdh1", "e-cad", "ecadherin"]
zeb_candidates  = ["zeb1", "zeb"]

Ecad_node = find_node_by_keywords(nodes_order, ecad_candidates)
ZEB_node  = find_node_by_keywords(nodes_order, zeb_candidates)

if not Ecad_node or not ZEB_node:
    raise RuntimeError(f"E-cad 또는 ZEB1 노드를 찾을 수 없음. nodes 예시: {nodes_order[:30]}")

print("Detected markers -> E-cad:", Ecad_node, ", ZEB1:", ZEB_node)

# 2) markers 변수 (노트북의 기존 markers 변수 대체)
markers = [Ecad_node, ZEB_node]

# 3) phenotype 정의: 논문 기준 (Epithelial / Mesenchymal / Hybrid)
# P1..P4은 모든 조합(1/0) 중 논문에서 말한 4개 조합(원하면 이름 바꿔도 됨)
phenotype = {
    'P_epithelial': {Ecad_node:1, ZEB_node:0},   # epithelial (E-cad + / ZEB1 -)
    'P_mesenchymal': {Ecad_node:0, ZEB_node:1},  # mesenchymal (E-cad - / ZEB1 +)
    'P_hybrid_high': {Ecad_node:1, ZEB_node:1},  # hybrid (E-cad + / ZEB1 +)
    'P_hybrid_low' : {Ecad_node:0, ZEB_node:0}   # hybrid (E-cad - / ZEB1 -)
}

# 4) phenotypeAnnot: 숫자 레이블 관례 (논문/분석 용도에 맞게)
# 여기선 epithelial = -1, mesenchymal = +1, hybrid들 = 0 (논문/시각화에서 자주 사용하는 구분)
phenotypeAnnot = {
    'P_epithelial': -1,
    'P_mesenchymal': 1,
    'P_hybrid_high': 0,
    'P_hybrid_low': 0
}

print("phenotype keys:", list(phenotype.keys()))
print("markers:", markers)
# --- 끝 ---


if 2**len(nodeList) >= 100000: num_init = 100000
else:  num_init = 2**len(nodeList)
initState = rand_initial_states(num_init, len(nodeList))

총 노드 수: 31
앞 40개 노드: ['AKT', 'AP1', 'ERK', 'Ecadherin', 'EpCAM', 'GLI', 'GRB2SOS', 'GSK3b', 'IKK', 'MDM2', 'MEK', 'MYC', 'NFKB', 'NOTCH', 'PI3K', 'RAF', 'RAS', 'SMAD23', 'SMAD4', 'Snail', 'TAK1', 'TGFBR', 'TGFb', 'THY1', 'Twist1', 'ZEB1', 'bcatenin', 'miR200', 'miR34', 'p38', 'p53']
Detected markers -> E-cad: Ecadherin , ZEB1: ZEB1
phenotype keys: ['P_epithelial', 'P_mesenchymal', 'P_hybrid_high', 'P_hybrid_low']
markers: ['Ecadherin', 'ZEB1']


### Node perturbation analysis

In [3]:
save_dir = './result/EMT/' 
save_perturbname = save_dir + 'EMT_single_simul_result.csv'

In [4]:
fix_dict = {'RAS':1}
# allsingles = [{n:1} for n in nodeList] + [{n:0} for n in nodeList]

In [5]:
perturb_p = pd.DataFrame([]) # average activities of the marker nodes
perturb_s = pd.DataFrame([]) # network stabiltiy 

fix_dict_tmp = copy.deepcopy(fix_dict) # fix_dict는 {'RAS':1} 인 상태

print(fix_dict_tmp) # 현재 적용할 고정 노드 출력

primes_new, pheno_df, att_ave_pd, attrs_dict = Simulation(fix_dict_tmp, primes, update_mode, initState, phenotype, phenotypeAnnot)   

att_ave_pd.to_csv("ras_on_att_ave_pd.csv")
pheno_df.to_csv("ras_on_pheno_df.csv")

{'RAS': 1}
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 48.76%)
    State Str:  1110011011111111111111111110010
  > Attractor ID 1 (Cyclic, Length 4, Basin 1.36%)
    Cycle Start Str:  1001100100100011100000001000101
       Step 1 Str:  1001100100100011100000001000101
       Step 2 Str:  1001000101100001100000000001101
       Step 3 Str:  0001000101100001100000000001100
       Step 4 Str:  0001100100100011100000001000100
  > Attractor ID 2 (Cyclic, Length 7, Basin 49.87%)
    Cycle Start Str:  1001100000110011100000011110000
       Step 1 Str:  1001100000110011100000011110000
       Step 2 Str:  1001100000110011100000011110001
       Step 3 Str:  1011000001110001100000000111101
       Step 4 Str:  0011000001100000101000001011100
       Step 5 Str:  1111100001010010101000011010100
       Step 6 Str:  110

### Node 하나씩 제거

In [6]:
# --------------- 로직만 없애는 시뮬레이션 프레임워크 ---------------
# 이 셀은 이전 코드 블록(RAS=1 기본 시뮬레이션) 다음에 실행
import time # time.time() 사용을 위해 필요
import copy # copy.deepcopy() 사용을 위해 필요
import pandas as pd # 결과 저장을 위해 필요
import os # 디렉토리 생성을 위해 필요

# 전역 변수로 이미 정의되어 있어야 할 변수들 확인:
# primes, nodes_order, update_mode, initState, phenotype, phenotypeAnnot, Simulation
fix_dict_for_single_run = {'RAS': 1} # (RAS=1만 고정된 상태에서 로직 제거 시뮬레이션)

print("\n--- 노드별 로직 제거(Logic Disruption) 분석 시작 ---")

# 결과 저장 디렉토리
results_dir_logic_disruption = './logic_disruption_results'
os.makedirs(results_dir_logic_disruption, exist_ok=True)

logic_disruption_results = []

# 원본 primes를 백업 (매 루프마다 초기 상태로 돌려야 하므로)
# primes는 `model_file = network_dir + 'EMT_Network.bnet'` 이후에 로드된 원본
original_primes_backup = copy.deepcopy(primes)

for node_to_disrupt_logic in nodes_order:
    # RAS 노드의 로직은 건드리지 않음 (이미 고정했거나 다른 중요한 기본 노드일 수 있으므로)
    # 필요하다면 다른 특정 노드도 제외 가능
    if node_to_disrupt_logic == 'RAS': # and 'RAS' in fix_dict_for_single_run:
        print(f"\n--- RAS 노드 로직은 건드리지 않음 (기본 고정값 사용) ---")
        continue

    # 1. primes를 현재 루프에 맞게 복사 (항상 원본 primes에서 시작)
    current_primes_for_disruption = copy.deepcopy(original_primes_backup)

    # 2. 해당 노드의 '로직(Boolean function)'을 제거
    #    -> 이 노드는 이제 다른 노드들의 영향을 받지 않고 '항상 1'을 출력한다고 가정
    #    사수님 의도에 따라: '실수로 고려 못했을 경우' = '기능적으로 비활성화'
    #    이를 모델에서는 그 노드가 항상 1 (ON) 값을 내게 함.
    constant_output_value = 1 # 로직 제거된 노드가 '항상 1'을 출력한다고 가정

    # current_primes_for_disruption에서 node_to_disrupt_logic의 Boolean 함수를 변경
    # primes[노드명] = [ [입력_노드_리스트], {(): 출력_값} ] 형태로 변경
    # 입력_노드_리스트를 빈 리스트 `[]`로 만들어 더 이상 입력에 의존하지 않게 하고,
    # {(): 출력_값}으로 항상 `constant_output_value`를 반환하게 함.
    current_primes_for_disruption[node_to_disrupt_logic] = [[], {(): constant_output_value}]
    
    print(f"\n--- 시뮬레이션: '{node_to_disrupt_logic}' 로직 제거 (항상 {constant_output_value} 출력) ---")
    print_node_logic(node_to_disrupt_logic, current_primes_for_disruption)

    # 3. Simulation 호출에 넘길 고정 노드 딕셔너리 생성
    #    RAS 고정 조건에 로직 제거된 노드도 그 고정 값으로 추가
    fix_dict_for_sim_call = copy.deepcopy(fix_dict_for_single_run) # {'RAS':1}
    fix_dict_for_sim_call.update({node_to_disrupt_logic: constant_output_value}) # 예: {'RAS':1, 'AKT':1}

    # Simulation 함수 호출
    try:
        primes_after_disruption, pheno_df_logic, att_ave_pd_logic, attrs_dict_logic = Simulation(
            fix_dict_for_sim_call,       # 이번 시뮬레이션의 고정 노드
            current_primes_for_disruption, # 로직 제거된 노드 정의가 바뀐 primes
            update_mode,
            initState,
            phenotype,
            phenotypeAnnot
        )
        
        # 결과 수집 (KO 시뮬레이션과 동일한 지표 사용)
        logic_disruption_results.append({
            'disrupted_node': node_to_disrupt_logic,
            'fixed_value_after_disruption': constant_output_value,
            'attractor_counts': len(attrs_dict_logic.get('attractors', {})),
            'phenotype_ratios': pheno_df_logic.T.to_dict('records')[0],
            'att_ave_pd_values': att_ave_pd_logic.T.to_dict('records')[0]
        })
    except Exception as e:
        print(f"에러 발생: '{node_to_disrupt_logic}' 로직 제거 시뮬레이션 중 - {e}")
        logic_disruption_results.append({
            'disrupted_node': node_to_disrupt_logic,
            'fixed_value_after_disruption': constant_output_value,
            'error': str(e),
            'attractor_counts': np.nan 
        })

# 결과를 Pandas DataFrame으로 정리
logic_disruption_df = pd.DataFrame(logic_disruption_results)
logic_disruption_df.set_index('disrupted_node', inplace=True)
logic_disruption_df.to_csv(os.path.join(results_dir_logic_disruption, 'node_logic_disruption_analysis_results.csv'))
print(f"\n노드 로직 제거 분석 결과 저장 완료: {os.path.join(results_dir_logic_disruption, 'node_logic_disruption_analysis_results.csv')}")


--- 노드별 로직 제거(Logic Disruption) 분석 시작 ---

--- 시뮬레이션: 'AKT' 로직 제거 (항상 1 출력) ---

--- 노드 'AKT'의 Boolean Function ---
  입력 노드들: 없음 (상수)
  진리표 (Truth Table): {(): 1}
  ==> 확인: 이 노드는 입력에 무관하게 항상 '1'을(를) 출력하는 상수 함수입니다 (로직 제거됨).
WARNING The state transition graph will consist of up to 2**31=2147483648 states, depending on the initial states.

--- [Simulation Function] Detected Attractors for Current Perturbation ---
  > Attractor ID 0 (Steady State, Basin 49.18%)
    State Str:  1110011011111111111111111110010
  > Attractor ID 1 (Cyclic, Length 4, Basin 1.08%)
    Cycle Start Str:  1001000101100001100000000001101
       Step 1 Str:  1001000101100001100000000001101
       Step 2 Str:  1001000101100001100000000001100
       Step 3 Str:  1001100100100011100000011000100
       Step 4 Str:  1001100100100011100000011000101
  > Attractor ID 2 (Cyclic, Length 7, Basin 49.74%)
    Cycle Start Str:  1001100000110011100000011110000
       Step 1 Str:  1001100000110011100000011110000
       Step 2 Str: